In [1]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(stringr))
library(parallel)
library(DESeq2)
library(hise)
source('/home//jupyter/BRI_Figures_Final_V2/helper_function/helper_function_IHA.r')

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The following object is masked from ‘package:tidyr’:

    expand


The following objects are masked from ‘package:dplyr’:

    first, rename


The following object is masked from

# Read Meta Data

In [2]:
meta_data=read.csv("/home/jupyter/BRI_Figures_Final_V2/Figure5/01_Frequency_Comparison/selected_samples_with_acutal_flu_year.csv")

# Perform Deseq2 on Year 1 Day 0 samples

In [3]:
filtered_gene_set<-read.csv("filtered_gene_Y2020-2021_Y2021-2022_D7.csv")

In [4]:
meta_data_subset=meta_data %>% filter(Flu_Day=='Day 7')

aggregated_count_file_list<-paste0("/home/jupyter/BRI_Figures_Final_V2/Dataset/scRNA/BRI/Aggregated_Count/Aggregated_Raw_Expression_of_Celltypes_by_Sample_AIFI_L3/",meta_data_subset$pbmc_sample_id,".csv")

df_list<-read_pseudobulk_expression(aggregated_count_file_list)

[1] "Total reading time: 15.158 seconds"
[1] "The length of the list matches the length of the input path."


In [5]:
celltypes <- unique(unlist(lapply(df_list, names)))
celltypes<-unique(sub(".*:", "", celltypes))

In [6]:
res_list<-mclapply(celltypes, function(celltype){
    print(celltype)
    celltype_list <- lapply(df_list, function(df) { df <- df[, grep(celltype, names(df), fixed = TRUE),drop=FALSE] })
    exp_matrix<-do.call(cbind,celltype_list)
    colnames(exp_matrix)<-sub(":.*", "", colnames(exp_matrix))
    rownames(meta_data_subset)<-meta_data_subset$pbmc_sample_id
    filtered_gene_set_filtered<-filtered_gene_set%>% filter(AIFI_L3==celltype)%>% select(gene) %>%pull()
    
    res=deseq2_analysis(exp_matrix,
                     meta_data=meta_data_subset,
                     filtered_gene_set=filtered_gene_set_filtered,
                     formula= ~  cohort.cohortGuid+subject.biologicalSex+CMV+Flu_Year,
                     comparisons=list(c("cohort.cohortGuid", "BR1", "BR2")),
                     celltype=celltype)
    return(res)
},mc.cores=60)

In [7]:
res_all<-do.call(rbind,res_list)

In [8]:
head(res_all)

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,contrast,Direction,gene,celltype
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1,0.4041447,0.213540577,0.3423002,0.62384007,0.53273262,0.9995678,cohort.cohortGuid,HigherInBR1,AL669831.5,ASDC
2,1.8930982,0.004231346,0.1970302,0.02147563,0.98286625,0.9995678,cohort.cohortGuid,HigherInBR1,NOC2L,ASDC
3,1.2124276,-0.208204059,0.3104029,-0.67075425,0.50237710,0.9995678,cohort.cohortGuid,HigherInBR2,ISG15,ASDC
4,2.4409849,0.160380580,0.1695837,0.94573132,0.34428565,0.9995678,cohort.cohortGuid,HigherInBR1,SDF4,ASDC
5,0.8935068,-0.031199783,0.2809857,-0.11103689,0.91158709,0.9995678,cohort.cohortGuid,HigherInBR2,B3GALT6,ASDC
6,2.3720525,0.316309879,0.1710183,1.84956791,0.06437585,0.9995678,cohort.cohortGuid,HigherInBR1,UBE2J2,ASDC


In [9]:
write.csv(res_all,"Deseq2_Result_Y2020-2021_Y2021-2022_D7.csv")